# Redo Big Run: apply small color 3-class CNN to mixed patches

This file is similar to DF_Filter.020
Use the CNN was trained on the obviously blue, green, black from DF_Filter.021      
Predict the 'mixed' patches i.e. not obviously blue, green, black from DF_Filter.021  
Generate csv file.

In [1]:
import time
import os
#  os.environ['CUDA_VISIBLE_DEVICES'] = "0"   # turns off GPU?
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
DIR_IMAGES_IN = "/home/jrm/Martinez/images/patches/"
#DIR_IMAGES_IN = "/home/jrm/Martinez/images/tinytest/"   # just for testing
PATTERN_PATCHES = "*.jpg"
OUTPUT_CSV = "predictions.csv"
PATCH_SIZE=224  # matches VGG
IMAGE_SIZE=(PATCH_SIZE,PATCH_SIZE)
DIR_MODELS = "/home/jrm/Martinez/models/"
FILE_MODEL = "CNN_SMALL_REDO"
filepath=DIR_MODELS+FILE_MODEL
print("Will use model from "+filepath)
cnn = keras.models.load_model(filepath)

Will use model from /home/jrm/Martinez/models/CNN_SMALL_REDO


In [3]:
def get_image_names(path,pattern):
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_PATCHES = get_image_names(DIR_IMAGES_IN,PATTERN_PATCHES)

In [4]:
normalization_layer = kl.Rescaling(1.0/255.0)

In [6]:
# With keras models, augmentation and dropout are inactive during prediction.
import csv
csvpath = DIR_IMAGES_IN+OUTPUT_CSV
with open(csvpath, 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',')
    start = time.time()
    for filename in FILENAMES_PATCHES:
        path = DIR_IMAGES_IN+filename
        img = keras.preprocessing.image.load_img(
            path, target_size=IMAGE_SIZE
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = normalization_layer(img_array)
        img_array = tf.expand_dims(img_array, 0)  # create tensor
        # TODO: consider using predict_classes() instead
        predictions = cnn.predict(img_array)
        score = predictions[0]  # TODO: explore other array slices
        blackscore,bluescore,greenscore = score
        if blackscore >= 0.99:
            word = "black"
        elif bluescore >= 0.99:
            word = "blue"
        elif greenscore >= 0.99:
            word = "green"
        else:
            word = "uncertain"
        datawriter.writerow( (filename, score[0], score[1], word) )
    end = time.time()
print("Elapsed time:",end-start)

Elapsed time: 34.316102504730225


## Results

78 black
688 green  
69 blue  
650 uncertain

TO DO: Adjust score thresholds because these numbers are too low. And why don't we have thousands?